In [ ]:
!pip install transformers datasets evaluate -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset, load_metric
from transformers import (
    BertTokenizerFast, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorForSeq2Seq
)
import torch
import numpy as np
import evaluate
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

## Load Data

In [ ]:
paws = load_dataset("paws", 'labeled_final')

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset paws downloaded and prepared to /root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
paws

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [ ]:
paws["train"][0]

{'id': 1,
 'sentence1': 'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .',
 'sentence2': 'In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .',
 'label': 0}

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(paws["train"])

,id,sentence1,sentence2,label
0,16924,Santa Ana Nopalucan is a municipality in Tlaxcala in south-eastern Mexico .,Santa Ana Nopalucan is a municipality in southeastern Tlaxcala in Mexico .,0
1,45906,Searching a binary search tree after a specific key can be programmed recursively or iteratively .,Searching a specific search tree for a binary key can be programmed recursively or iteratively .,0
2,13159,Red Bank is located on the 11th Congressional District and is part of the 4th State Legislative District in New Jersey .,Red Bank is located in the 11th Congressional District and is part of New Jersey 's 4th state legislative district .,1
3,4642,"Face of Evil is a 1996 TV movie starring Jeanelle Polk as Shawnee Smith , Perry King as Russell Polk and Darcy Palmer as his daughter Tracey Gold .","Face of Evil is a TV movie from 1996 with Tracey Gold as Darcy Palmer , Perry King as Russell Polk and Shawnee Smith as the daughter Jeanelle Polk .",0
4,10845,"Two of these four hostels are located in Delhi , one each in Pune and Pimpri near Mumbai .","Two of these four hostels are located in Mumbai , each in Delhi and Pimpri , near Pune .",0
5,1481,"The Franciscan is the literary magazine , with events , annual articles , submissions by students and staff and photographs .","The Franciscan is the literary magazine , with events , annual articles , submissions from students and from staff , and photographs .",1
6,6705,"The heart of Sharjah is home to several galleries and museums , including the Museum of Cultural Heritage , which gives a charming insight into the great traditions of the past .","The Heart Of Sharjah houses several galleries , and museums including the charming heritage museum which gives a great insight into the cultural traditions of the past .",0
7,19802,"A costumed carnival showing `` chirigotas '' , large groups that sing in a humorous way of current events .","A large carnival features `` chirigotas '' , costumed groups that sing of current events in a humorous way .",0
8,45843,"The average body length is 50 centimeters , but the maximum length is 25 centimetres .","The average body length is 50 centimeters , but the maximum length is 25 centimeters .",1
9,23885,The other European powers were no longer prepared to oppose a new French expansion and were willing to form alliances to accept such a thing .,The other European powers were no longer disposed to accept a new French expansion and were prepared to form alliances to oppose such a thing .,0


## Download model

In [ ]:
model_checkpoint = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Tokenizing

In [ ]:
# Tokenize the input text
def tokenize_function(example):
    tokens = tokenizer(
        example['sentence1'], 
        example['sentence2'], 
        truncation=True
    )
    tokens["labels"] = example["label"]
    return tokens

tokenized_dataset = paws.map(tokenize_function, batched=True)

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
})

## Modeling

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
# print("we are using",device)

In [ ]:
example = tokenized_dataset['train'][0]
print(example.keys())

dict_keys(['id', 'sentence1', 'sentence2', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenized_dataset['train'][0]

{'id': 1,
 'sentence1': 'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .',
 'sentence2': 'In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .',
 'label': 0,
 'input_ids': [101,
  1999,
  3000,
  1010,
  1999,
  2255,
  29185,
  1010,
  2002,
  10082,
  2777,
  1996,
  2394,
  6059,
  1010,
  9473,
  16215,
  16901,
  5302,
  11715,
  1010,
  4851,
  2032,
  2005,
  1037,
  12293,
  2000,
  2709,
  2000,
  2563,
  2083,
  3885,
  1012,
  102,
  1999,
  2255,
  29185,
  1010,
  2002,
  10082,
  2777,
  2007,
  1996,
  2394,
  6059,
  1010,
  9473,
  16215,
  16901,
  5302,
  11715,
  1010,
  1999,
  3000,
  1010,
  1998,
  2356,
  2032,
  2005,
  1037,
  12293,
  2000,
  2709,
  2000,
  3885,
  2083,
  2563,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,

In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] in paris, in october 1560, he secretly met the english ambassador, nicolas throckmorton, asking him for a passport to return to england through scotland. [SEP] in october 1560, he secretly met with the english ambassador, nicolas throckmorton, in paris, and asked him for a passport to return to scotland through england. [SEP]'

In [ ]:
tokenized_dataset['train'].features


{'id': Value(dtype='int32', id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['0', '1'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Value(dtype='int64', id=None)}

In [ ]:
# metric = evaluate.load("accuracy", "f1")
metric = evaluate.load("glue", "qqp")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_total_limit=1,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],    
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.387300,0.383597,0.838375,0.820591
1000,0.191200,0.270683,0.910875,0.903895
1500,0.169200,0.236448,0.925000,0.917966
2000,0.120600,0.254533,0.926625,0.919490
2500,0.054900,0.341472,0.925625,0.919453
3000,0.057200,0.341230,0.933000,0.926736
3500,0.023500,0.337865,0.932125,0.925728


TrainOutput(global_step=3860, training_loss=0.18076690008062773, metrics={'train_runtime': 3557.6654, 'train_samples_per_second': 69.429, 'train_steps_per_second': 1.085, 'total_flos': 9782040968931060.0, 'train_loss': 0.18076690008062773, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.34122997522354126,
 'eval_accuracy': 0.933,
 'eval_f1': 0.9267359212684527,
 'eval_runtime': 38.0319,
 'eval_samples_per_second': 210.35,
 'eval_steps_per_second': 52.587,
 'epoch': 5.0}

In [ ]:
model.push_to_hub("harouzie/bert-base-paws")
tokenizer.push_to_hub("harouzie/bert-base-paws")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harouzie/bert-base-paws/commit/b4150748f790fb5348887570eea6bb5a23058406', commit_message='Upload tokenizer', commit_description='', oid='b4150748f790fb5348887570eea6bb5a23058406', pr_url=None, pr_revision=None, pr_num=None)

### building pipeline on our model

In [ ]:
model_checkpoint = "harouzie/bert-base-paws"

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, padding="max_length")

In [ ]:
sample_test = paws["test"][:10]
sample_sents = list(zip(sample_test["sentence1"], sample_test["sentence2"]))

In [ ]:
sample_sents[:3]

[('This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .',
  'This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .'),
 ('His father emigrated to Missouri in 1868 but returned when his wife became ill and before the rest of the family could also go to America .',
  'His father emigrated to America in 1868 , but returned when his wife became ill and before the rest of the family could go to Missouri .'),
 ('In January 2011 , the Deputy Secretary General of FIBA Asia , Hagop Khajirian , inspected the venue together with SBP - President Manuel V. Pangilinan .',
  'In January 2011 , FIBA Asia deputy secretary general Hagop Khajirian along with SBP president Manuel V. Pangilinan inspected the venue .')]

In [ ]:
classifier(sample_sents)

[{'label': 'not_paraphrased', 'score': 0.603177011013031},
 {'label': 'paraphrased', 'score': 0.5388592481613159},
 {'label': 'not_paraphrased', 'score': 0.9675319194793701},
 {'label': 'paraphrased', 'score': 0.5814346075057983},
 {'label': 'not_paraphrased', 'score': 0.7182698845863342},
 {'label': 'not_paraphrased', 'score': 0.9836347103118896},
 {'label': 'paraphrased', 'score': 0.746323823928833},
 {'label': 'paraphrased', 'score': 0.5502285957336426},
 {'label': 'not_paraphrased', 'score': 0.7866140604019165},
 {'label': 'not_paraphrased', 'score': 0.7991027235984802}]